In [2]:
!nvidia-smi

Fri Aug 15 18:04:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 33.7 MB/s eta 0:00:00


In [5]:
!pip install --upgrade accelerate
!pip uinstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pagasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [11]:
dataset_samsun = load_dataset("knkarthick/samsum")
dataset_samsun

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [17]:
def convert_example_to_features(batch):
    # Replace None with empty string so tokenizer doesn't break
    dialogues = [d if isinstance(d, str) else "" for d in batch["dialogue"]]
    summaries = [s if isinstance(s, str) else "" for s in batch["summary"]]

    # Tokenize inputs (dialogue)
    model_inputs = tokenizer(
        dialogues,
        max_length=1024,
        truncation=True
    )

    # Tokenize targets (summary)
    labels = tokenizer(
        text_target=summaries,
        max_length=1024,
        truncation=True
    )

    # Add labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [22]:
dataset_samsun_pt = dataset_samsun.map(convert_example_to_features, batched =True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [23]:
dataset_samsun_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [24]:
#Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_pagasus)

In [29]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus-samsun",
    num_train_epochs=1,
    learning_rate=5e-5,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000000,
    gradient_accumulation_steps=16
)


In [51]:
trainer = Trainer(
    model = model_pagasus,
    args = trainer_args,
    tokenizer = tokenizer,
    data_collator = seq2seq_data_collator,
    train_dataset = dataset_samsun_pt["train"],
    eval_dataset = dataset_samsun_pt["validation"]
)

/tmp/ipython-input-1293660131.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [32]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.672800,1.476692


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=921, training_loss=1.7225352743161229, metrics={'train_runtime': 2698.4208, 'train_samples_per_second': 5.459, 'train_steps_per_second': 0.341, 'total_flos': 5538138246070272.0, 'train_loss': 1.7225352743161229, 'epoch': 1.0})

In [33]:
#Evaluation
"""
def generate_batch_size_chunk(list_of_elements, batch_size):
  for i in range(0,len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

  def caluculate_metric_on_train_ds(dataset, metric,
                                    model,
                                    tokenizer,
                                    batch_size = 16,
                                    device=device,
                                    column_text="article",
                                    column_summary = "highlights"):

    article_batches = list(generate_batch_size_chunk(dataset[column_text], batch_size))
    target_batches = list(generate_batch_size_chunk(dataset[column_summary],batch_size))

    for article_batch, target_batch in tdqm(
        zip(article_batches,target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch,max_length = 1024,truncation = True,
                           padding = "max_length",
                           return_tensors="pt")
        summaries = model.generate(input_idx = input["input_ids"].to(device),
                                   attention_mask = inputs["attention_mask"].to(device),
                                   length_penalty = 0.8,num_beams=8, max_length= 128)
        decoded_summary = [tokenizer.decode(s,skip_special_tokens = True,clean_up_tokenization_spaces=True)
        for s in summaries]

        decoded_summary = [d.replace(""," ") for d in decoded_summary]

        metric.add_batch(predections = decoded_summary, references = target_batch)
    score = metric.compute()
    return score
"""
from tqdm import tqdm

def generate_batch_size_chunk(list_of_elements, batch_size):
    """Yield successive batch_size chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def calculate_metric_on_train_ds(dataset, metric,
                                 model,
                                 tokenizer,
                                 batch_size=16,
                                 device="cuda",
                                 column_text="dialogue",
                                 column_summary="summary"):

    article_batches = list(generate_batch_size_chunk(dataset[column_text], batch_size))
    target_batches = list(generate_batch_size_chunk(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        # Tokenize inputs
        inputs = tokenizer(article_batch,
                           max_length=1024,
                           truncation=True,
                           padding="max_length",
                           return_tensors="pt")

        # Generate summaries
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8,
                                   num_beams=8,
                                   max_length=128)

        # Decode
        decoded_summary = [
            tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for s in summaries
        ]

        decoded_summary = [d.replace(""," ") for d in decoded_summary]
        # Add batch to metric
        metric.add_batch(predictions=decoded_summary, references=target_batch)

    # Compute final score
    score = metric.compute()
    return score






In [38]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00


In [42]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8a2f0ef2aca7622e0cd51d29470490a10d38ebbc3f1ae805db4088ac77d189e0
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [43]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load("rouge")

In [47]:
score = calculate_metric_on_train_ds(
    dataset_samsun["test"], rouge_metric, trainer.model, tokenizer, batch_size=2,
    column_text="dialogue",
    column_summary="summary"
)

# Now score[rn] is already a float
rouge_dict = {rn: score[rn] for rn in rouge_names}

import pandas as pd
pd.DataFrame(rouge_dict, index=["pegasus"])


  0%|          | 2/410 [00:08<28:05,  4.13s/it]


KeyboardInterrupt: 

In [52]:
model_pagasus.save_pretrained("pegasus-samsun-model")

In [53]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [55]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")


In [56]:
#Predction
gen_kwargs = {"length_penalty":0.8, "num_beams":8,"max_length":128}

sample_text = dataset_samsun["test"][0]["dialogue"]
refrence = dataset_samsun["test"][0]["summary"]

pipe = pipeline("summarization", model = "pegasus-samsun-model", tokenizer=tokenizer)

print("Dialogue:")
print(sample_text)

print("summarization")
print(refrence)

Device set to use cuda:0


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
summarization
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [1]:
import shutil
import os
from google.colab import files

# Create a temp folder to hold both model and tokenizer
os.makedirs("export_model", exist_ok=True)

# Copy model and tokenizer folders into it
shutil.copytree("pegasus-samsun-model", "export_model/pegasus-samsun-model")
shutil.copytree("tokenizer", "export_model/tokenizer")

# Zip everything
shutil.make_archive("pegasus_samsun_full", 'zip', "export_model")

# Download the zip
files.download("pegasus_samsun_full.zip")


FileNotFoundError: [Errno 2] No such file or directory: 'pegasus-samsun-model'